## 使用 torch.autograd 自动区分#
创建时间： 2021 年 2 月 10 日 |上次更新时间：2024 年 1 月 16 日 |上次验证： Nov 05， 2024

在训练神经网络时，最常用的算法是 反向传播 。在此算法中，参数（模型权重）根据损失函数相对于给定参数的梯度进行调整。

为了计算这些梯度，PyTorch 有一个名为 torch.autograd 的内置微分引擎。它支持任何计算图的梯度自动计算。

考虑最简单的单层神经网络，其中包含输入 x、参数 w 和 b 以及一些损失函数。可以在 PyTorch 中按以下方式定义它：



In [ ]:
import torch

x = torch.ones(5)  # input tensor
y = torch.zeros(3)  # expected output
print(f"x: {x},\ny: {y}")

x: tensor([1., 1., 1., 1., 1.]),
y: tensor([0., 0., 0.])


: 

In [ ]:

w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
print(f"w: {w},\nb: {b}")

w: tensor([[ 1.2828,  0.0374, -0.3987],
        [-0.3069, -0.0065,  0.2534],
        [-0.0264,  1.8649, -0.0342],
        [-1.2278,  0.2466, -1.1349],
        [ 0.7971, -0.3001, -0.3748]], requires_grad=True),
b: tensor([ 0.2504, -0.5290,  0.3083], requires_grad=True)


## 计算梯度#
为了优化神经网络中参数的权重，我们需要 计算我们的损失函数关于参数的导数， 即，我们需要 
∂loss/∂w 并且 ∂loss/∂b 在 一些 固定 值 下 x 和 y。为了计算这些导数，我们调用 loss.backward（）， 然后从 w.grad 和 学士 ：

In [ ]:
# 前向传播
z = torch.matmul(x, w)+b
print(f"z: {z}")
# 计算损失
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)
print(f"loss: {loss}")

# 反向传播，计算梯度
loss.backward()
print(f"w.grad: {w.grad}")
print(f"b.grad: {b.grad}")

# 初始化梯度优化器，用来决定如何更新参数。
# SGD：随机梯度优化
# (w, b)：传入要优化的参数
# lr：学习率
optimizer = torch.optim.SGD((w, b), lr=0.01)
# 更新参数
optimizer.step()
print(f"w: {w}")
print(f"b: {b}")

w.grad.zero_()
b.grad.zero_()

z: tensor([-6.6991, -6.6982, -6.7060], grad_fn=<AddBackward0>)
loss: 0.0012288093566894531
w.grad: tensor([[0.0004, 0.0004, 0.0004],
        [0.0004, 0.0004, 0.0004],
        [0.0004, 0.0004, 0.0004],
        [0.0004, 0.0004, 0.0004],
        [0.0004, 0.0004, 0.0004]])
b.grad: tensor([0.0004, 0.0004, 0.0004])
w: tensor([[ 0.0377, -1.2983, -1.2866],
        [-1.5520, -1.3422, -0.6345],
        [-1.2716,  0.5292, -0.9221],
        [-2.4729, -1.0891, -2.0228],
        [-0.4481, -1.6357, -1.2627]], requires_grad=True)
b: tensor([-0.9947, -1.8646, -0.5796], requires_grad=True)


tensor([0., 0., 0.])

## 可选读数：张量梯度和雅可比积

In [1634]:
inp = torch.eye(4, 5, requires_grad=True)
out = (inp+1).pow(2).t()

out.backward(torch.ones_like(out), retain_graph=True)
print(f"First call\n{inp.grad}")
out.backward(torch.ones_like(out), retain_graph=True)
print(f"\nSecond call\n{inp.grad}")
inp.grad.zero_()
out.backward(torch.ones_like(out), retain_graph=True)
print(f"\nCall after zeroing gradients\n{inp.grad}")

First call
tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.]])

Second call
tensor([[8., 4., 4., 4., 4.],
        [4., 8., 4., 4., 4.],
        [4., 4., 8., 4., 4.],
        [4., 4., 4., 8., 4.]])

Call after zeroing gradients
tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.]])


请注意，当我们使用相同的参数第二次向后调用时，梯度的值是不同的。发生这种情况是因为在进行向后传播时，PyTorch 会累积梯度 ，即计算的梯度值被添加到 grad 属性。 如果要计算正确的梯度，则需要将 grad 归零 属性之前。在现实生活中的训练中， 优化器可以帮助我们做到这一点。